In [5]:
# this notebook iterate over neuron 4D files (already masked and filtered) 
# and stream-pipeline the defined analysis steps 
# and save the output into one dataframe per neuron with the following columns
# Output_name : XXXXXXXXX define
# output_cols: XXXXXX define

In [6]:
data_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/'
save_path = '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/output/'

##
stab_limit = 5 #(No. of timepoints for stable branches calculation)
start_age = 37
subtype='A' #define the subtype to be used in DGI func

analysis_type = '3D' #define whether the analysis_type is 2D or 3D
entry_points = {'211104_P36_N1': (0,25,220,230),
                '211104_P36_N3': (0,27,110,250),
                '211107_P36_N2': (0,27,225,235),
                '211107_P36_N3': (0,27,225,235),
                '211113_P36_N1': (0,26,200,220),
                '211114_P36_N3': (0,28,260,200),
                '211121_P36.5_N3': (0,34,195,165),
                '220127_P36_N1': (0,19,210,220),
                '220209_P36_N1': (0,27,225,235),
                '211011_P37_N1': (0,38,150,200),
                '211104_P36_N2': (0,16,240,240), 
                '211107_P36_N1': (0,33,185,250),
                '211113_P36_N2': (0,28,195,270), 
                '211114_P36_N1': (0,44,225,170), 
                '211114_P36_N2': (0,43,280,250), 
                '220209_P36_N2': (0,27,290,245),    
                '211121_P36.5_N1': (0,24,210,260), 
                '211121_P36.5_N2': (0,31,235,240),
                '211028_P36_N1': (0,25,210,240),
                '211025_P36.5_N1': (0,25,210,240),
                '211025_P36.5_N2': (0,25,210,240),
                '211025_P36.5_N3': (0,25,210,240),
                '211025_P36.5_N4': (0,25,210,240)}


if save_path[-1] != '/':
    save_path += '/'
if data_path[-1] != '/':
    data_path += '/'

In [7]:
import os
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
import math
from skimage.measure import regionprops
from read_roi import read_roi_zip as col_zip
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import os
from matplotlib import colors
import seaborn as sns
import cv2
from skimage import measure, io, morphology, transform
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import glob
from sklearn import decomposition
from scipy import spatial
import pickle
import itertools

import matplotlib as mpl
from cycler import cycler

%matplotlib inline

In [8]:
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if '_P' in f]
N_folders = sorted(N_folders)
N_folders[:5]

['211028_P36_N1',
 '211104_P36_N2',
 '211107_P36_N1',
 '211113_P36_N2',
 '211114_P36_N1']

In [9]:
for i, fold in tqdm(enumerate(N_folders)):
    # try:
    #     for n in [1]:
            entry_point = entry_points[fold]
            start_im = re.search('P(.*)_N', fold)
            start_im = float(start_im.group(1))
            start_ana = int((start_age - start_im)/0.25)
            files_list = datautils.get_file_names(data_path+fold, group_by='', 
                                                order=True, nested_files=False, 
                                                criteria='')
            neu_path = [f for f in files_list if 'clahe' in f.lower()][0]
            neuron = tif.imread(neu_path)
            mask_path = [f for f in files_list if 'mask' in f.lower()][0]
            mask = tif.imread(mask_path)
            neuron[mask==0] = 0
            neuron = neuron[start_ana:]
            neuron[neuron != 0] = 1
            lifetimes = analysis.cal_lifetimes(neuron, save=False)
            stable_neuron = analysis.stable_N(lifetimes, stab_limit=stab_limit) 
            newly_formed = lifetimes.copy()
            newly_formed[newly_formed != 5] = 0
            neu_2D = neuron.max(axis=1)
            stable_2D = stable_neuron.max(axis=1)
            newly_2D = newly_formed.max(axis=1)
            if analysis_type=='3D':
                image = neuron
                stable_im = stable_neuron
            elif analysis_type=='2D':
                image = neu_2D
                stable_im = stable_2D
            else:
                print('the analysis type is not recognized')
                break
            output_results = analysis.N_volume(image, stable_im, normalize=False, 
                                                start_t=start_age, plot=False, 
                                                save=False, save_path=save_path, 
                                                save_file='Vol_'+fold)
            DGIs = analysis.calculate_DGI(entry_point, image, 
                                subtype=subtype,
                                start_t=start_age, 
                                save=True, save_path='', 
                                save_file='')
            output_results = pd.merge(output_results, DGIs, on='timepoints')
            area, perimeter, density, circularity = analysis.cal_circularity(neu_2D)
            output_results['circularity'] = circularity[:]
            output_results['density'] = density[:]
            output_results['area'] = area[:]
            output_results['perimeter'] = perimeter[:]
            # yx_stable = []
            # yx_new = []
            yx_all = []
            surface_area = []     
            for t, stack in enumerate(neu_2D[:],0):
                mo = cv2.moments(stack,binaryImage=False)
                # mo_newly = cv2.moments(newly_2D[t],binaryImage=False)
                # yx_stable.append((mo['m01']/mo['m00'], mo['m10']/mo['m00']))
                # yx_new.append((mo_newly['m01']/mo_newly['m00'], mo_newly['m10']/mo_newly['m00']))
                yx_all.append((mo['m01']/mo['m00'], mo['m10']/mo['m00']))
                verts, faces, normals, values = measure.marching_cubes(neuron[t], step_size=2, spacing=(0.4,0.076,0.076))
                surface_area.append(measure.mesh_surface_area(verts, faces))
            # output_results['stab_cent'] = yx_stable
            # output_results['newly_cent'] = yx_new
            output_results['all_cent'] = yx_all
            output_results['surface_area'] = surface_area
            output_file = save_path+fold+'_s'+subtype+'_output.csv'
            output_file = output_file.replace('.tif','')
            output_results.to_csv(output_file, sep=';')  
    # except:
    #     pass          

0it [00:00, ?it/s]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211028_P36_N1/211028_P36_N1_R225_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211028_P36_N1/211028_P36_N1_R225_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211028_P36_N1/211028_P36_N1_R225_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211028_P36_N1/211028_P36_N1_R225_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211028_P36_N1/211028_P36_N1_CropRoi.roi']


100%|██████████| 85/85 [01:30<00:00,  1.06s/it]
1it [01:48, 108.53s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_ColROIs.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_5D_new.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211104_P36_N2/211104_P36_N2_R270_5D.tif']


100%|██████████| 85/85 [02:59<00:00,  2.12s/it]
2it [05:18, 167.93s/it]

...

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_clahe1.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_JE_mask.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_FlipV_R277_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N1/211114_P36_N1_CropRoi.roi']


100%|██████████| 89/89 [03:23<00:00,  2.28s/it]
5it [14:49, 192.77s/it]

first 5 files
['/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R277_JE_seg.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_clahe.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_ColRois.zip', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_FlipV_R272_5D.tif', '/home/tli_project/Desktop/Data/T4/neurons/SubtypeA/211114_P36_N2/211114_P36_N2_CropRoi.roi']


5it [14:58, 179.65s/it]


IndexError: list index out of range